# TOP 1000 filmes e séries de TV - IMDb

## Projeto Final - Banco de Dados I

- Escolha um dataset de sua preferência (Kaggle, UCI,...);
  - https://www.kaggle.com/datasets;
  - https://archive.ics.uci.edu/datasets;
  - https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html;
  - https://dados.gov.br/;
- Coloque-o no postgres; 
- Usando o Jupyter notebook, faça uma análise descritiva;
    - Qual as tipagens das colunas?;
    - Quantos nulos temos em cada coluna?;
    - Quantos valores únicos temos nas colunas categoricas?;
    - ...;
- Ainda no Jupyter, faça 10 perguntas ao seu dataset e responda-as com querys;
    - Qual o top 10 vendas?;
    - Qual o top 5 clientes?;
    - Qual medida é minoria?;
    - ...;

### Integrantes

- Guilherme Araújo Mendes de Souza
- Laura Busin Campos

### Dataset utilizado
- **IMDB Movies Dataset:** https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

### Importação e verificação da base

#### Bibliotecas

In [41]:
import pandas as pd
from sqlalchemy import create_engine, text

In [232]:
imdb = pd.read_csv('imdb_top_1000.csv')

#### Verificação do estado da base

In [233]:
imdb.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [234]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


#### Tratamento do tipo de algumas colunas

In [235]:
imdb['Gross'] = imdb['Gross'].str.replace(',', '').replace('', '0').astype(float)

In [236]:
imdb['Runtime'] = imdb['Runtime'].str.replace(' min', '').replace('', '0').astype(int)

#### Separação da coluna de Gênero em outras colunas

In [238]:
imdb["Genre"] = imdb["Genre"].str.split(', ')
imdb = imdb.join(imdb['Genre'].apply(lambda x: pd.Series(x)).add_prefix('Genre_'))

#### Tratamento das colunas com nulos

In [239]:
colunas_com_nulo = ['Certificate', 'Genre_0', 'Genre_1', 'Genre_2']

for coluna in colunas_com_nulo:
    imdb[coluna].fillna('', inplace=True)

In [240]:
imdb['Meta_score'].fillna(0, inplace=True)
imdb['Gross'].fillna(0, inplace=True)

#### Dropando colunas indesejadas

In [241]:
imdb.drop(columns = ["Genre", "Poster_Link"], inplace = True)

#### Verificando o estado da base depois do tratamento

In [242]:
imdb.head()

,Series_Title,Released_Year,Certificate,Runtime,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Genre_0,Genre_1,Genre_2
0,The Shawshank Redemption,1994,A,142,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0,Drama,,
1,The Godfather,1972,A,175,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0,Crime,Drama,
2,The Dark Knight,2008,UA,152,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0,Action,Crime,Drama
3,The Godfather: Part II,1974,A,202,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0,Crime,Drama,
4,12 Angry Men,1957,U,96,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0,Crime,Drama,


In [243]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Certificate    1000 non-null   object 
 3   Runtime        1000 non-null   int32  
 4   IMDB_Rating    1000 non-null   float64
 5   Overview       1000 non-null   object 
 6   Meta_score     1000 non-null   float64
 7   Director       1000 non-null   object 
 8   Star1          1000 non-null   object 
 9   Star2          1000 non-null   object 
 10  Star3          1000 non-null   object 
 11  Star4          1000 non-null   object 
 12  No_of_Votes    1000 non-null   int64  
 13  Gross          1000 non-null   float64
 14  Genre_0        1000 non-null   object 
 15  Genre_1        1000 non-null   object 
 16  Genre_2        1000 non-null   object 
dtypes: float64(3), int32(1), int64(1), object(12)
memory 

#### Salvando o Data Frame em um csv

In [244]:
imdb.to_csv('imdb_top_1000_tratado.csv', index=False)

### Criação da tabela no PostgreSQL para fazer a importação do csv criado

CREATE TABLE ada.imdb(
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SERIAL
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,nm_filme&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(100)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,ano&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(4)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,certificado&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,duracao&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;INTEGER
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,classificacao_imdb&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FLOAT
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,resumo&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(1000)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,pontuacao&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FLOAT
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,diretor&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,votos&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;INTEGER
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,bilheteria&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FLOAT
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,genero1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,genero2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,genero3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,PRIMARY KEY (id)
<br>);

#### Configurando a conexão com o banco de dados

Para fazer a conexão com o seu banco de dados, salve um arquivo com o nome configSGBD.txt fora da pasta do projeto com as seguintes inforamções e fotrmatação:
- {"SGBD": "xxxxx", "USER": "xxxxx", "SENHA": "xxxxx", "HOST": "xxxxx", "DATABASE": "xxxxx"}

In [51]:
# Puxando a informação do arquivo de configuração do SGBD

config = ''
with open('..\configSGBD.txt', 'r') as file:
    for line in file:
        config += line.strip('')

In [52]:
# Transformando em um dicionário

dict_config = eval(config)

In [53]:
# Fazendo a conexão com o SGBD

engine = create_engine(f"{dict_config['SGBD']}://{dict_config['USER']}:{dict_config['SENHA']}@{dict_config['HOST']}:5432/{dict_config['DATABASE']}")
conn = engine.connect()

In [47]:
# Verificando se a conexão deu certo

query = """SELECT * FROM ada.imdb LIMIT 5"""

pd.read_sql_query(text(query), con = conn)

,id,nm_filme,ano,certificado,duracao,classificacao_imdb,resumo,pontuacao,diretor,estrela1,estrela2,estrela3,estrela4,votos,bilheteria,genero1,genero2,genero3
0,1,The Shawshank Redemption,1994,A,142,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0,Drama,None,None
1,2,The Godfather,1972,A,175,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0,Crime,Drama,None
2,3,The Dark Knight,2008,UA,152,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0,Action,Crime,Drama
3,4,The Godfather: Part II,1974,A,202,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0,Crime,Drama,None
4,5,12 Angry Men,1957,U,96,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0,Crime,Drama,None


### Análise Descritiva

### 10 perguntas sobre o dataset

Algumas ideis de perguntas que eu tive:
- Top 10 filmes com maior bilheteria
- Top 10 filmes com maior classificação IMDb
- Top 10 gêneros com maior bilheteria
- Top 10 diretores com maior bilheteria